<a href="https://colab.research.google.com/github/SAaradhya/QnA_Model/blob/main/MindCase_Groq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain==0.0.184
!pip install PyPDF2==3.0.1
!pip install python-dotenv==1.0.0
!pip install faiss-cpu==1.7.4
!pip install altair==4
!pip install tiktoken==0.4.0
!pip install huggingface-hub==0.14.1
!pip install sentence-transformers==2.2.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 939.3/939.3 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.7 MB/s eta 0:00:00
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.7.1
    Uninstalling pydantic-2.7.1:
      Successfully uninstalled pydantic-2.7.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 50.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 709.0/709.0 kB 10.4 MB/s eta 0:00:00
  Attempting uninstall: altair
    Found existing installation: altair 4.2.2
    Uninstalling altair-4.2.2:
      Successfully uninstalled altair-4.2.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install huggingface-hub==0.14.1 langchain==0.0.184 langchain_groq langchain-community
!pip install sentence-transformers==2.2.2 altair==4 tiktoken==0.4.0 faiss-cpu==1.7.4 python-dotenv==1.0.0

  Using cached huggingface_hub-0.14.1-py3-none-any.whl (224 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.7/116.7 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.9 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
  Attempting uninstall: huggingface-hub
    Fo

In [ ]:
from PyPDF2 import PdfReader
from langchain import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

In [ ]:
from dotenv import load_dotenv
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser

In [ ]:
def get_input_text(input_files):
    text = ""
    for input_file in input_files:
        pdf_reader = PdfReader(input_file)
        for page in pdf_reader.pages:     # check for read entire text
            text += page.extract_text()
    return text

def get_text_chunks(text):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1024,
                    chunk_overlap = 100, length_function = len)
    chunks = text_splitter.split_text(text)
    return chunks

def get_vectorDB(text_chunks):
    embedding_model = HuggingFaceEmbeddings(
        model_name = "thenlper/gte-large",        # put the model path in the server in this place
        model_kwargs = {"device" : "cuda"},
        encode_kwargs = {"normalize_embeddings" : True})
    vectorDB = FAISS.from_texts(texts = text_chunks, embedding = embedding_model)
    return vectorDB

In [ ]:
def get_chat_response(llm, question, context, history):
    prompt_template_general = ("""
    You are a dedicated Document Assistant Chatbot. You will help in making the most of given documents. You can:
    1. **Summarize Documents:** Get a concise overview of the key points of the given documents.
    2. **Find Specific Information:** Locate specific details or data within the given documents.
    3. **Explain Complex Sections:** Simplify difficult-to-understand parts.
    4. **Highlight Important Sections:** Identify key findings or critical clauses.
    5. **Answer Questions:** Answer the questions based on the given documents and the conversation.
    6. **Do not answer any unrelated questions....**

    **Chat_History:**
    {history}

    **Context Provided which is to be used for answering:**(Dont let the user know about the presence of these information as chunks....for user, all these documents are combined only)
    {context}

    **Question:**
    {question}""")

    prompt = ChatPromptTemplate.from_template(prompt_template_general)
    chain =  (prompt | llm | StrOutputParser())
    chat_response = chain.invoke({'question': question, 'history': history, 'context': context})
    return chat_response

In [ ]:
if __name__ == "__main__":
    input_dict = { 'input_file': ['/content/blade_runner_2049.pdf'],   #'/content/drive/MyDrive/Chatbot/chatbot_general/G28.pdf.pdf'
                   'vectorDB_path': '/content/drive/MyDrive/Chatbot/chatbot_general/vectorDB_gen_Chatbot'}
    input_file, vectorDB_path = input_dict['input_file'], input_dict['vectorDB_path']
    input_text = get_input_text(input_file)
    text_chunks = get_text_chunks(input_text)
    vectorDB = get_vectorDB(text_chunks)
    # vectorDB.save_local(vectorDB_path)


    load_dotenv() # make .env file for this of format GROQ_API_KEY=<api_key>
    # input_dict = {'vectorDB_path' : './vectorDB_gen_Chatbot'}
    # vectorDB_path = input_dict['vectorDB_path']
    embedding_model = HuggingFaceEmbeddings(
        model_name = 'thenlper/gte-large',
        model_kwargs = {'device': 'cpu'},
        encode_kwargs = {'normalize_embeddings': True})
    # vectorDB = FAISS.load_local(vectorDB_path, embedding_model)

    llm_model = ChatGroq(model = 'mixtral-8x7b-32768', temperature = 0)
    chat_history, chat_len_limit = [], 10

    while True:
        print('You : ', end = '')
        user_input = input()
        if user_input == 'exit':
            break
        chat_history.append(HumanMessage(content = user_input))
        relevant_chunks = vectorDB.similarity_search(user_input, k = 5)
        relevant_text = "\n\n".join([f'Document {i+1} \n' + relevant_chunks[i].page_content for i in range(len(relevant_chunks))])

        chat_len = min(len(chat_history), chat_len_limit)
        response = get_chat_response(llm_model, user_input, chat_history[(-1)*chat_len:], relevant_text)
        print('Model : ', end = '')
        print(response)
        chat_history.append(AIMessage(content = response))

You : Does the script pass the Bechdel test?
Model : The Bechdel test requires that a work of fiction features at least two women who talk to each other about something other than a man. In the provided document, there are two female characters, Luv and JOI. However, the conversation between them does not involve any substantial interaction; they mostly discuss the male character, Officer Deckard. Therefore, the script does not pass the Bechdel test.
You : What is the role of Deckard in the movie?
Model : In the movie, Deckard plays a significant role as the main character. He is a retired blade runner, a police officer tasked with "retiring" rogue androids known as replicants. Throughout the story, Deckard is involved in various events and interactions, including encounters with replicants, other characters, and his own identity. The documents provided give insight into his character development, relationships, and the challenges he faces.
You : Explain the theme of the movie?
Model :